# Getty Query Tutorial (Date of Death)
In this notebook, we perform the similar hunger-for-knowledge procedure for date of death in Wikidata and Getty Vocabulary. The main difference is that we generalize the query for single property to the property chain, this is, the path in kgtk.

In [1]:
import os
import json
import pandas as pd

from kgtk.functions import kgtk, kypher

## Step 0: Set up environment paths
Here we set needed Graph files and desired output files to environment paths.

In [2]:
# We will define environment variables to hold the full paths to the files as we will use them in the shell commands
kgtk_environment_variables = []

# Folder where database files store
data_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
os.environ['DATABASE'] = data_path
kgtk_environment_variables.append('DATABASE')

# Wikidata (all is much less than claims)
os.environ['WIKIDATA'] = data_path + "claims.tsv"
kgtk_environment_variables.append('WIKIDATA')

# Label file of Wikidata
os.environ['KGTK_LABEL_FILE'] = data_path + "labels.en.tsv"
kgtk_environment_variables.append('KGTK_LABEL_FILE')

# P31
os.environ['P31'] = data_path + "P31.tsv"
kgtk_environment_variables.append('P31')

# P279star
os.environ['P279STAR'] = data_path + "P279star.tsv"
kgtk_environment_variables.append('P279STAR')

# Folder of ULAN
ulan_path = data_path + "gvp/ULAN/"
os.environ['ULAN'] = ulan_path
kgtk_environment_variables.append('ULAN')

# Full file provided by Getty 
# Full file is much truncated in this process thus we will use concat files below instead.
# os.environ['ULAN_FULL'] = ulan_path + "full.tsv"
# kgtk_environment_variables.append('ULAN_FULL')

# File concatenated by used explicit files: ULAN Subjects, AgentMap, Biographies
# The suffix ID means we performed add-id operation to the graph to meet the requirement of paths command.
ulan_concat_path = ulan_path + "ulan.bio.concat.id.tsv"
os.environ['ULAN_BIO_CONCAT_ID'] = ulan_concat_path
kgtk_environment_variables.append('ULAN_BIO_CONCAT_ID')

# ULAN-Wikidata alignment file, maps ULAN ID to Wikidata Qnode
ulan_wikialign_path = ulan_path + "wiki.align.tsv"
os.environ['ULAN_ALIGN'] = ulan_wikialign_path
kgtk_environment_variables.append('ULAN_ALIGN')

# Output
output_path = data_path + "gvp/dod_output/"
if not os.path.exists(output_path):
    os.mkdir(output_path)
os.environ['OUTPUT'] = output_path
kgtk_environment_variables.append('OUTPUT')

# Each file will be explained later in the procedure
output_names = {
    "wiki_dod": "wiki.dod.tsv",
    "wiki_unknown": "wiki.unknown.tsv",
    "pairs": "pairs.tsv",
    "paths": "paths.tsv",
    "paths_label": "paths.label.tsv",
    "getty_dod": "getty.dod.tsv",
    "getty_error": "getty.error.tsv",
    "getty_mapped": "getty.mapped.tsv",
    "getty_agree": "getty.agree.tsv",
    "correct": "correct.tsv",
    "unknown": "unknown.tsv"
}

for key, value in output_names.items():
    variable = key.upper()
    os.environ[variable] = os.path.join(output_path, value)
    kgtk_environment_variables.append(variable)

for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

DATABASE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
WIKIDATA: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/claims.tsv"
KGTK_LABEL_FILE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/labels.en.tsv"
P31: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P31.tsv"
P279STAR: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P279star.tsv"
ULAN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/"
ULAN_BIO_CONCAT_ID: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/ulan.bio.concat.id.tsv"
ULAN_ALIGN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/wiki.align.tsv"
OUTPUT: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/dod_output/"
WIKI_DOD: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/dod_output/wiki.dod.tsv"
WIKI_UNKNOWN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/dod_output/wiki.unknown.tsv"
PAIRS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/dod_output/pairs.tsv"
PATHS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/g

## Step 1: Query Wikidata
Query Wikidata for PoB for people that have an ULAN ID (92k people)

In [3]:
# count total poeple
# ULAN ID
kgtk("""
    query -i $ULAN_ALIGN 
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_627_c1.""node1"")"
0,92353


In [4]:
# Qnode
kgtk("""
    query -i $ULAN_ALIGN 
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c1.""node2"")"
0,92210


In [3]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $WIKIDATA
        --match 'w: (ulanid)-[]->(qnode), 
                 a: (qnode)-[:P570]->(dod)' 
        --return 'distinct ulanid as node1, "P570" as label, dod as node2'
        --limit 10
    / add-labels
    """)

CPU times: user 40 ms, sys: 42.9 ms, total: 82.9 ms
Wall time: 1min 31s


,node1,label,node2,label;label
0,ulan:500023949,P570,^1945-02-08T00:00:00Z/11,'date of death'@en
1,ulan:500123827,P570,^1954-09-07T00:00:00Z/11,'date of death'@en
2,ulan:500072302,P570,^1957-12-20T00:00:00Z/11,'date of death'@en
3,ulan:500091345,P570,^1954-12-23T00:00:00Z/11,'date of death'@en
4,ulan:500020713,P570,^1935-05-30T00:00:00Z/11,'date of death'@en
5,ulan:500160869,P570,^1981-08-22T00:00:00Z/11,'date of death'@en
6,ulan:500108146,P570,^1588-01-01T00:00:00Z/9,'date of death'@en
7,ulan:500108146,P570,^1586-01-01T00:00:00Z/9,'date of death'@en
8,ulan:500113074,P570,^1884-07-27T00:00:00Z/11,'date of death'@en
9,ulan:500094949,P570,^2013-05-03T00:00:00Z/11,'date of death'@en


In [4]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $WIKIDATA
        --match 'w: (ulanid)-[]->(qnode), 
                 a: (qnode)-[:P570]->(dod)' 
        --return 'distinct ulanid as node1, "P570" as label, dod as node2'
        -o $WIKI_DOD
    """)

CPU times: user 13.8 ms, sys: 13.8 ms, total: 27.6 ms
Wall time: 17.1 s


## Step 2: Record Known and Unknown

In [5]:
# count known ULAN IDs
kgtk("""
    query -i $WIKI_DOD
        --match '(ulanid)-[]->(dod)' 
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_666_c1.""node1"")"
0,70938


In [4]:
# count known Qnodes
kgtk("""
    query -i $WIKI_DOD $ULAN_ALIGN
        --match 'd: (ulanid)-[]->(nation), a: (ulanid)-[]->(qnode)' 
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c2.""node2"")"
0,70837


Filter unknow in ULAN-Wikidata alignment

In [6]:
%%time
kgtk("""
    ifnotexists -i $ULAN_ALIGN 
        --filter-on $WIKI_DOD 
        --input-keys node1 
        --filter-keys node1 
        -o $WIKI_UNKNOWN
    """)

CPU times: user 6.23 ms, sys: 8.93 ms, total: 15.2 ms
Wall time: 4.59 s


In [7]:
# count unknown ULAN IDs
kgtk("""
    query -i $WIKI_UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_667_c1.""node1"")"
0,21415


In [3]:
# count unknown Qnodes
kgtk("""
    query -i $WIKI_UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_667_c1.""node2"")"
0,21373


## Step 3: Query Getty
Based on the known PoB for people, query Getty. Before we perform single property query (we can consider it as 1 hop paths query), this time, we generalize to paths between known pairs (Subjects, Objects). Similarily we will compute the distributions of found paths and pick the top 1 path (property chain). 

### 3-1: Build Pairs from Wikidata Results 
Pairs file is needed for performing kgtk paths operation as the query objects.

In [8]:
%%time
kgtk("""
    query -i $WIKI_DOD
        --match '(ulanid)-[]->(dod)'
        --return 'distinct ulanid as source, kgtk_stringify(kgtk_date_year(dod)) as target'
        --limit 200
        -o $PAIRS
    """)

CPU times: user 6.82 ms, sys: 10.5 ms, total: 17.4 ms
Wall time: 4.54 s


In [9]:
!head $PAIRS | column -ts $'\t'

source          target
ulan:500023949  "1945"
ulan:500123827  "1954"
ulan:500072302  "1957"
ulan:500091345  "1954"
ulan:500020713  "1935"
ulan:500160869  "1981"
ulan:500108146  "1588"
ulan:500108146  "1586"
ulan:500113074  "1884"


### 3-2: Perform Paths Mapping
This step is the core step of this query and it is also the bottleneck of running this procedure.

In [10]:
%%time
kgtk("""
    paths --max_hops 5
        --path-file $PAIRS
        --path-mode NONE 
        --path-source source
        --path-target target
        -i $ULAN_BIO_CONCAT_ID
        --statistics-only
        -o $PATHS
    """)

CPU times: user 136 ms, sys: 58.4 ms, total: 195 ms
Wall time: 3min 28s


In [11]:
!head $PATHS

node1	label	node2	id
p0	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p0-0-0
p0	1	ulan:500023949-agent:gvp:biographyNonPreferred:ulan_bio:4000061262	p0-1-1
p0	2	ulan_bio:4000061262:gvp:estEnd:nodeBKhLqNtUciRjAx7zs7VoBJ-236988	p0-2-2
p0	3	nodeBKhLqNtUciRjAx7zs7VoBJ-236988:gvp:structured_value:"1945"	p0-3-3
p1	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p1-0-4
p1	1	ulan:500023949-agent:gvp:biographyNonPreferred:ulan_bio:4000061263	p1-1-5
p1	2	ulan_bio:4000061263:gvp:estEnd:nodeBKhLqNtUciRjAx7zs7VoBJ-236990	p1-2-6
p1	3	nodeBKhLqNtUciRjAx7zs7VoBJ-236990:gvp:structured_value:"1945"	p1-3-7
p2	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p2-0-8


### 3-3: Select the Top 1 Property Chain
Here we use Pandas to sort the top 1 property chain. 

In [12]:
# load paths.tsv file
paths = pd.read_csv(os.environ['PATHS'], sep='\t')
# process property: since paths command generate id as node2, we need to further extract property from it
paths.node2 = paths.node2.apply(lambda x: ':'.join(x.split(':')[2:4]) if len(x.split(':')) > 4 else ':'.join(x.split(':')[1:3]))
# save processed property
paths.to_csv(os.environ['PATHS_LABEL'], sep='\t', index=False)
# join property chain for each found path
paths_concat = paths.groupby(paths['node1']).agg({'node2': lambda x: ' '.join(list(x))})
# sort the top 
paths_concat.value_counts()#.head(1)

node2                                                               
foaf:focus gvp:biographyNonPreferred gvp:estEnd gvp:structured_value    203
foaf:focus gvp:biographyPreferred gvp:estEnd gvp:structured_value       145
dtype: int64

### 3-4: Query Getty Using the Property Chain
Query Getty for the values of the unknown people, with the found property chain: `(ulanid)-[foaf:focus]->()-[gvp:biographyNonPreferred]->()-[gvp:estEnd]->()-[gvp:structured_value]->(String of Birth Year)`

In [3]:
%%time
kgtk("""
    query -i $WIKI_UNKNOWN $ULAN_BIO_CONCAT_ID
        --match 'unknown: (ulanid)-[]->(), 
                 ulan: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(dod)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "gvp:estEnd" AND
                 p3.label = "gvp:structured_value"'
        --return 'distinct ulanid as node1, "P570" as label, printf("^%s-01-01T00:00:00Z/9", kgtk_unstringify(dod)) as node2'
        --limit 10
    / add-labels
    """)

CPU times: user 48.6 ms, sys: 38.3 ms, total: 87 ms
Wall time: 1min 24s


,node1,label,node2,label;label
0,ulan:500000007,P570,^1616-01-01T00:00:00Z/9,'date of death'@en
1,ulan:500000009,P570,^1630-01-01T00:00:00Z/9,'date of death'@en
2,ulan:500000012,P570,^2049-01-01T00:00:00Z/9,'date of death'@en
3,ulan:500000015,P570,^1829-01-01T00:00:00Z/9,'date of death'@en
4,ulan:500000019,P570,^1520-01-01T00:00:00Z/9,'date of death'@en
5,ulan:500000024,P570,^1546-01-01T00:00:00Z/9,'date of death'@en
6,ulan:500000031,P570,^1530-01-01T00:00:00Z/9,'date of death'@en
7,ulan:500000039,P570,^2069-01-01T00:00:00Z/9,'date of death'@en
8,ulan:500000042,P570,^0140-01-01T00:00:00Z/9,'date of death'@en
9,ulan:500000048,P570,^1859-01-01T00:00:00Z/9,'date of death'@en


In [4]:
%%time
kgtk("""
    query -i $WIKI_UNKNOWN $ULAN_BIO_CONCAT_ID
        --match 'unknown: (ulanid)-[]->(), 
                 ulan: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(dod)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "gvp:estEnd" AND
                 p3.label = "gvp:structured_value"'
        --return 'distinct ulanid as node1, "P570" as label, dod as node2'
        -o $GETTY_DOD
    """)

CPU times: user 8.2 ms, sys: 13.4 ms, total: 21.6 ms
Wall time: 14.8 s


Count how many new results (count in ULAN ID) we found

In [5]:
kgtk("""
    query -i $GETTY_DOD
        --match '(ulanid)-[]->(dod)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_7163_c1.""node1"")"
0,19674


## Step 4: Record New Results

Map back to Wikidata

In [6]:
%%time
kgtk("""
    query -i $GETTY_DOD $ULAN_ALIGN
        --match 'g: (ulanid)-[p]->(dod), 
                 w: (ulanid)-[]->(qnode)'
        --return 'distinct qnode as node1, p.label as label, dod as node2'
        -o $GETTY_MAPPED
    """)

CPU times: user 4.71 ms, sys: 11.5 ms, total: 16.2 ms
Wall time: 4.43 s


Count how many new results (count in Qnode) we found

In [7]:
kgtk("""
    query -i $GETTY_MAPPED
        --match '(qnode)-[]->(dod)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_7164_c1.""node1"")"
0,19654


## Step 5: Validate with Wikidata Constraints
Since

In [8]:
dod_df = pd.read_csv(os.environ["GETTY_DOD"], sep='\t')
dod_df['node2'] = dod_df['node2'].astype('int')
dod_df = dod_df.loc[(dod_df['node2'] >= -9999) & (dod_df['node2'] <= 2021)]
dod_df.to_csv(os.environ["CORRECT"], sep='\t', index=False)

Count in Qnode how many correct results we get:

In [9]:
kgtk("""
    query -i $CORRECT $ULAN_ALIGN
        --match 'c: (ulanid)-[]->(), w: (ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c2.""node2"")"
0,5995


## Step 6: Record Unknown after Query
We record unknowns after query Getty.

In [10]:
kgtk("""
    ifnotexists -i $WIKI_UNKNOWN \
        --filter-on $CORRECT \
        --input-keys node1 \
        --filter-keys node1
        -o $UNKNOWN
    """)

In [11]:
# count in ULAN IDs
kgtk("""
    query -i $UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_7166_c1.""node1"")"
0,15414


In [12]:
# count in Qnodes
kgtk("""
    query -i $UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_7166_c1.""node2"")"
0,15379


# Step 7: Comparison of the Known Values
How often do the results from two knowledge graphs agree

In [13]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $ULAN_BIO_CONCAT_ID
        --match 'w: (ulanid)-[]->(), 
                 u: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(dod)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "gvp:estEnd" AND
                 p3.label = "gvp:structured_value"'
        --return 'distinct ulanid as node1, "P570" as label, dod as node2'
        -o $GETTY_AGREE
    """)

CPU times: user 2.16 ms, sys: 16.2 ms, total: 18.3 ms
Wall time: 9.26 s


Count agree in Qnode:

In [14]:
%%time
kgtk("""
    query -i $WIKI_DOD $GETTY_AGREE $ULAN_ALIGN
        --match 'd: (ulanid)-[]->(dod), g: (ulanid)-[]->(yod), a: (ulanid)-[]->(qnode)'
        --where 'kgtk_stringify(kgtk_date_year(dod)) = yod'
        --return 'count(distinct qnode)'
    """)

CPU times: user 8.29 ms, sys: 11.4 ms, total: 19.7 ms
Wall time: 4.47 s


,"count(DISTINCT graph_627_c3.""node2"")"
0,55588


Count value per node:

In [15]:
nodes = kgtk("""
    query -i $GETTY_AGREE $ULAN_ALIGN 
        --match 'g: (ulanid)-[]->(), w: (ulanid)-[]->(qnode)'
        --return 'count(distinct qnode) as node'
    """)

In [16]:
import subprocess

values = subprocess.check_output("wc -l < $GETTY_AGREE", shell=True)
values = values.decode("utf-8").strip()
values = int(values) - 1

In [17]:
values / nodes.iloc[0]['node']

1.0013372825534361